In [1]:
from unityagents import UnityEnvironment
import numpy as np

In [2]:
env = UnityEnvironment(file_name='Reacher.app')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [3]:
#Get the Default Brain
brain_name = env.brain_names[0]
print(brain_name)

ReacherBrain


In [4]:
brain = env.brains[brain_name]
print(brain)

Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [5]:
#Reset the Environment
env_info = env.reset(train_mode=True)[brain_name]

#Number of Agents
num_agents = len(env_info.agents)
print('Number of Agents:', num_agents)

#Size of Each Action
action_size = brain.vector_action_space_size
print('Size of Each Action:', action_size)

#Examine the State Space
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} Agents. Each Observes a State with Length: {}'.format(states.shape[0], state_size))
print('The State for the First Agent Looks Like:', states[0])

Number of Agents: 1
Size of Each Action: 4
There are 1 Agents. Each Observes a State with Length: 33
The State for the First Agent Looks Like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  5.75471878e+00 -1.00000000e+00
  5.55726671e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -1.68164849e-01]


In [6]:
env_info = env.reset(train_mode=False)[brain_name]
states = env_info.vector_observations
scores = np.zeros(num_agents)

while True:
    actions = np.random.randn(num_agents, action_size)
    actions = np.clip(actions, -1, 1)
    env_info = env.step(actions)[brain_name]
    next_states = env_info.vector_observations
    rewards = env_info.rewards
    dones = env_info.local_done
    scores += env_info.rewards
    states = next_states
    if np.any(dones):
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 0.1699999962002039


In [7]:
env.close()

In [1]:
import gym
import torch as T
import numpy as np

In [2]:
class ActorCritic(T.nn.Module):
    def __init__(self):
        super(ActorCritic, self).__init__()
        self.transform = T.nn.Linear(8, 128)
        self.act_layer = T.nn.Linear(128, 4) # Action layer
        self.val_layer = T.nn.Linear(128, 1) # Value layer
        self.log_probs = []
        self.state_vals = []
        self.rewards = []

    def forward(self, state):
        state = T.from_numpy(state).float()
        state = T.nn.functional.relu(self.transform(state))
        state_value = self.val_layer(state)

        act_probs = T.nn.functional.softmax(self.act_layer(state))
        act_dist = T.distributions.Categorical(act_probs)
        action = act_dist.sample()

        self.log_probs.append(act_dist.log_prob(action))
        self.state_vals.append(state_value)

        return action.item()

    def computeLoss(self, gamma=0.99):
        rewards = []
        discounted_reward = 0
        for reward in self.rewards[::-1]:
            discounted_reward = reward + gamma * discounted_reward
            rewards.insert(0, discounted_reward)

        rewards = T.tensor(rewards)
        rewards = (rewards - rewards.mean()) / (rewards.std())

        loss = 0
        for log_probability, value, reward in zip(
            self.log_probs, self.state_vals, rewards):
            advantage = reward - value.item()
            act_loss = -log_probability * advantage
            val_loss = T.nn.functional.smooth_l1_loss(value, reward)
            loss += (act_loss + val_loss)

        return loss

    def clear(self):
        del self.log_probs[:]
        del self.state_vals[:]
        del self.rewards[:]

In [3]:
np.random.seed(42)

In [4]:
env = gym.make("LunarLander-v2")

policy = ActorCritic()
optimizer = T.optim.Adam(policy.parameters(), lr=0.02, betas=(0.9, 0.999))

In [5]:
render = True
np.random.seed(0)
running_reward = 0
for i in np.arange(0, 10000):
    state = env.reset()
    for t in range(10000):
        action = policy(state)
        state, reward, done, _ = env.step(action)
        policy.rewards.append(reward)
        running_reward += reward
        if render and i > 1000:
            #env.render()
            pass
        if done:
            break
    print("Episode {}\tReward: {}".format(i, running_reward))

    # Updating the policy
    optimizer.zero_grad()
    loss = policy.computeLoss(0.99).float()
    loss.backward
    optimizer.step()
    policy.clear()

    if i % 20 == 0:
        running_reward = running_reward / 20
        running_reward = 0

/Users/oluwaseyiawoga/opt/miniconda2/envs/drlnd/lib/python3.6/site-packages/ipykernel_launcher.py:16: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  app.launch_new_instance()
/Users/oluwaseyiawoga/opt/miniconda2/envs/drlnd/lib/python3.6/site-packages/ipykernel_launcher.py:40: UserWarning: Using a target size (torch.Size([])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.


Episode 0	Reward: -19.530120740207337
Episode 1	Reward: -106.48525133131827
Episode 2	Reward: -347.33905417607167
Episode 3	Reward: -545.5610030436301
Episode 4	Reward: -687.9402631199421
Episode 5	Reward: -823.7514980310119
Episode 6	Reward: -1091.934241898779
Episode 7	Reward: -1221.316136658783
Episode 8	Reward: -1419.8411762976427
Episode 9	Reward: -1519.9627051088992
Episode 10	Reward: -1636.099477299539
Episode 11	Reward: -1868.631282970872
Episode 12	Reward: -1988.893548061565
Episode 13	Reward: -2059.2110481864274
Episode 14	Reward: -2159.5524905050424
Episode 15	Reward: -2454.6888237332632
Episode 16	Reward: -2821.3827613110698
Episode 17	Reward: -2945.1761322073367
Episode 18	Reward: -3166.1812864217945
Episode 19	Reward: -3609.811554299518
Episode 20	Reward: -3772.0104759490723
Episode 21	Reward: -178.60116862119173
Episode 22	Reward: -257.12674922801045
Episode 23	Reward: -371.9950582398401
Episode 24	Reward: -670.8373602385619
Episode 25	Reward: -738.3813762496615
Episode 

Episode 215	Reward: -2719.7605504149965
Episode 216	Reward: -2841.490820320693
Episode 217	Reward: -3034.5609844720243
Episode 218	Reward: -3310.649194300424
Episode 219	Reward: -3458.1544346640317
Episode 220	Reward: -3703.949346512306
Episode 221	Reward: -227.18694814833916
Episode 222	Reward: -375.54454105318314
Episode 223	Reward: -559.7686761326112
Episode 224	Reward: -885.7529944091226
Episode 225	Reward: -1057.8802023576704
Episode 226	Reward: -1141.5884400955015
Episode 227	Reward: -1204.4403559207665
Episode 228	Reward: -1290.231517394974
Episode 229	Reward: -1509.7957146270962
Episode 230	Reward: -1571.1771088445305
Episode 231	Reward: -1644.7920929802497
Episode 232	Reward: -1895.754549860395
Episode 233	Reward: -2022.31021116184
Episode 234	Reward: -2143.8351348378696
Episode 235	Reward: -2421.216695216045
Episode 236	Reward: -2703.865964345552
Episode 237	Reward: -2818.9449278513594
Episode 238	Reward: -3089.099651518393
Episode 239	Reward: -3372.667572043255
Episode 240	R

KeyboardInterrupt: 

[ 0.00610914  1.419455    0.6187742   0.37931958 -0.00707219 -0.1401617
  0.          0.        ]


<bound method EnvSpec.make of EnvSpec(LunarLanderContinuous-v2)>

Size of Each Action: 2
There are 8 Agents. Each Observes a State with Length: 8
The State for the First Agent Looks Like: 8


# Source - The Reinforcement Learning Workshop PacktPub

In [16]:
from unityagents import UnityEnvironment
import numpy as np
env = UnityEnvironment(file_name='Reacher.app')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


In [17]:
#env = UnityEnvironment(file_name='Reacher.app')
brain_name = env.brain_names[0]
brain = env.brains[brain_name]
env_info = env.reset(train_mode=True)[brain_name]

In [18]:
#Reset the Environment
env_info = env.reset(train_mode=True)[brain_name]

#Number of Agents
num_agents = len(env_info.agents)
print('Number of Agents:', num_agents)

#Size of Each Action
action_size = brain.vector_action_space_size
print('Size of Each Action:', action_size)

#Examine the State Space
states = env_info.vector_observations
state_size = states.shape[1]
print('There are {} Agents. Each Observes a State with Length: {}'.format(states.shape[0], state_size))
print('The State for the First Agent Looks Like:', states[0])

Number of Agents: 1
Size of Each Action: 4
There are 1 Agents. Each Observes a State with Length: 33
The State for the First Agent Looks Like: [ 0.00000000e+00 -4.00000000e+00  0.00000000e+00  1.00000000e+00
 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -1.00000000e+01  0.00000000e+00
  1.00000000e+00 -0.00000000e+00 -0.00000000e+00 -4.37113883e-08
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00 -6.30408478e+00 -1.00000000e+00
 -4.92529202e+00  0.00000000e+00  1.00000000e+00  0.00000000e+00
 -5.33014059e-01]


In [19]:
agent = Agent(
    alpha=0.25, beta=0.25, inp_dimensions=[33], tau=0.1,
    env=env, bs=64, l1_size=4, l2_size=3, nb_actions=4)




num_agents = 1
brain_name = env.brain_names[0]
#env_info = env.reset(train_mode=False)[brain_name]
env_info = env.reset(train_mode=True)[brain_name]
states = env_info.vector_observations
scores = np.zeros(num_agents)
actions = np.random.randn(1, 4)

for x in range(0,100):    
    while True:
        observation=env_info.vector_observations
        actions = agent.select_action(observation)
        #actions = np.clip(actions, -1, 1) 
        env_info = env.step(actions)[brain_name]
        next_states = env_info.vector_observations
        rewards = env_info.rewards
        dones = env_info.local_done
        scores += env_info.rewards
        states = next_states
        if np.any(dones):
            break
    print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x33 and 8x400)

In [20]:
env_info = env.reset(train_mode=True)[brain_name]
observation=env_info.vector_observations

In [21]:
agent = Agent(
    alpha=0.000025, beta=0.00025, inp_dimensions=[33], tau=0.001,
    env=env, bs=64, l1_size=400, l2_size=300, nb_actions=4)



In [22]:
actions = agent.select_action(observation)

In [23]:
actions

array([[ 0.02404302,  0.02123703, -0.00587684, -0.0319232 ]],
      dtype=float32)

In [24]:
num_agents = 1
brain_name = env.brain_names[0]
#env_info = env.reset(train_mode=False)[brain_name]
states = env_info.vector_observations
scores = np.zeros(num_agents)


In [25]:
while True:
    #actions = np.random.randn(num_agents, action_size)
    env_info = env.reset(train_mode=True)[brain_name]
    observation=env_info.vector_observations
    actions = agent.select_action(observation)
    env_info = env.step(actions)[brain_name]
    next_states = env_info.vector_observations
    rewards = env_info.rewards
    dones = env_info.local_done
    scores += env_info.rewards
    states = next_states
    if np.any(dones):
        break
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

KeyboardInterrupt: 

In [26]:
print('Total score (averaged over agents) this episode: {}'.format(np.mean(scores)))

Total score (averaged over agents) this episode: 0.0


In [20]:
env.brain_names

['ReacherBrain']

In [26]:
env.local_done

[924]